Let $\Sigma_t$ denote portfolio value under our managment
$$\Sigma_t = n_i(t) \cdot I_i(t),$$
where $n_i(t)$ -- position in the instrument $i$, $I_i(t)$ -- its present value

$$\dot\Sigma(t) = \dot n_i I_i + n_i \dot I_i$$

Hence we must implement Instruments' dynamics and portolio strategy to control $n_i$ 

In [1]:
import numpy as np
import pandas as pd
from lptools.utils import Adapter, Runner
from lptools.clients import Client
from lptools.instruments import UniPool, Bond, Portfolio, Position, Cash, Perpetual
from lptools.instruments import Holder, PerpetHedger
from copy import copy
from lptools.solver import Solver

In [2]:
zcurve = pd.read_csv('zcyc_rates.csv', index_col=0, parse_dates=True, infer_datetime_format=True, na_filter=False,)

a = Adapter(
        pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
        symbol="ETHUSDT",
        zcurve=zcurve,
        base_interval='1d'
)

#response = adapter(300)
#response.to_csv('saved_response.csv')

In [3]:
response = pd.read_csv('saved_response.csv', index_col=0, infer_datetime_format=True)
init_state = response.iloc[0]

In [4]:
solver = Solver(response)
results = solver.solve(t = 20,
             delta1=.1, q1=.95,
             delta2=.2, q2=.95,
             verbose=False)

print(results)
pl, pu = map(lambda x: x*init_state.token0Price, results.x)

           constr: [array([0.]), array([0.]), array([0.])]
 constr_violation: 0.0
              fun: -20.926666666666666
            maxcv: 0.0
          message: 'Optimization terminated successfully.'
             nfev: 712
              nit: 13
          success: True
                x: array([0.34238569, 1.96125647])


1. Hodl

In [5]:
perpetual = Position(tag='perpet', instrument=Perpetual())
bond = Position(tag='bond', instrument=Bond(init_state))

positions = [perpetual, bond]
balancer = Holder(positions, initial_amounts=[1 / init_state.token0Price, -1])
portfolio_hodl = Portfolio(balancer, positions=positions)

2. Passive pool

In [6]:

pool = Position(tag='pool', instrument=UniPool(init_state , price_lower=pl, price_upper=pu, fees=.005))
bond = Position(tag='bond', instrument=Bond(init_state))


positions = [pool, bond]
balancer = Holder(positions, initial_amounts=[1,-1])
portfolio_pooler = Portfolio(balancer, positions=positions)

3. Hedging

In [7]:
pool = Position(tag='pool', instrument=UniPool(init_state , price_lower=pl, price_upper=pu, fees=.005))
bond = Position(tag='bond', instrument=Bond(init_state))
perpetual = Position(tag='perpet', instrument=Perpetual())

positions = [pool, perpetual, bond]

delta =  pool.instrument.L / init_state.token0Price**.5
balancer = PerpetHedger(
    positions,
    initial_amounts=[1, - delta, delta * init_state.token0Price - 1 ],
    rebalancing_interval= 1
)
portfolio_perpet = Portfolio(balancer, positions)

In [8]:
runner = Runner(response,[portfolio_hodl, portfolio_pooler, portfolio_perpet])
runner.run()

In [9]:
runner.summary

,total_value,discounted_value
portfolio 0,0.285268,0.274328
portfolio 1,0.101759,0.097259
portfolio 2,0.026239,0.024306


In [11]:
runner.logs(1)

,pool_value,pool_amount,bond_value,bond_amount,payments,transaction_costs,total_value
0,1.000013,1,1.000139,-1,0.000000e+00,-0.02,-0.000127
1,0.998211,1,1.000279,-1,9.056849e-17,0.00,-0.002068
2,1.054052,1,1.000418,-1,1.034291e-15,0.00,0.053633
3,1.057450,1,1.000558,-1,1.399983e-16,0.00,0.056892
4,1.065387,1,1.000697,-1,2.054000e-16,0.00,0.064690
...,...,...,...,...,...,...,...
413,1.075175,1,1.059425,-1,2.783312e-16,0.00,0.015750
414,1.082566,1,1.059572,-1,1.220083e-16,0.00,0.022994
415,1.108335,1,1.059720,-1,1.324533e-16,0.00,0.048615
416,1.127702,1,1.059868,-1,1.025739e-16,0.00,0.067835


In [12]:
runner.logs(2)

,pool_value,pool_amount,perpet_value,perpet_amount,bond_value,bond_amount,payments,transaction_costs,total_value
0,1.000013,1,3496.292879,-0.000143,1.000139,-0.5,0.000000e+00,-0.015001,-0.000070
1,0.998211,1,3483.701459,-0.000143,1.000279,-0.5,-1.406753e-06,-0.000009,-0.001034
2,1.054052,1,3884.370431,-0.000136,1.000418,-0.5,8.526746e-05,-0.000264,0.026817
3,1.057450,1,3909.453752,-0.000135,1.000558,-0.5,-9.141232e-07,-0.000017,0.028446
4,1.065387,1,3968.366567,-0.000134,1.000697,-0.5,1.167594e-05,-0.000039,0.032345
...,...,...,...,...,...,...,...,...,...
413,1.075175,1,4041.612472,-0.000133,1.059425,-0.5,-3.851266e-05,-0.000329,0.007875
414,1.082566,1,4097.371402,-0.000132,1.059572,-0.5,2.214724e-06,-0.000037,0.011497
415,1.108335,1,4294.761309,-0.000129,1.059720,-0.5,7.882936e-06,-0.000126,0.024308
416,1.127702,1,4446.166582,-0.000127,1.059868,-0.5,5.789726e-06,-0.000095,0.033917
